# IMPORTS

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error


## Etapa 4

Nesta secção será realizado arquiteturas de deep learning como DNN e o CNN onde o objetivo será otmixar hiperparametros destas mesmas arquiteturas tendo em conta que o custo computacional é elevado.

In [4]:
path1="X_train.csv"
path2="y_train.csv"
path3="X_val.csv"
path4="y_val.csv"
path5="X_test.csv"
path6="y_test.csv"

X_train=pd.read_csv(path1)
y_train=pd.read_csv(path2)
X_val=pd.read_csv(path3)
y_val=pd.read_csv(path4)
X_test=pd.read_csv(path5)
y_test=pd.read_csv(path6)

Para a arquitetura de DNN, a estratégia passou por permitir ter uma abordagem rectangular onde o numero de neurónio nas diferentes hidden layers é o mesmo, ou uma abordagem cónica onde em cada hidden layer o numero de neurónios é metade do numero de neurónios da camada anterior.

Realizamos esta abordagem apenas para um máximo de 3 camadas (hidden) tendo em conta que computacionalemente é pesado, mas tendo perfeita noção de que poderiamos aumentar o numero de camadas.

Além disso verificamos o comportamento com taza de dropout de 0.2 e 0.5 bem como eraling stopping de 3.

In [3]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV

def create_model(neurons=1, optimizer='adam', layer_type='rectangular', dropout_rate=0.0, num_layers=2):
    model = Sequential()

    if layer_type == 'rectangular':
        for _ in range(num_layers):
            model.add(Dense(neurons, input_dim=X_train.shape[1], activation='relu'))
            # Adicionar dropout se o dropout_rate for maior que 0
            if dropout_rate > 0:
                model.add(Dropout(dropout_rate))
    elif layer_type == 'conic':
        for _ in range(num_layers):
            model.add(Dense(neurons, input_dim=X_train.shape[1], activation='relu'))
            if dropout_rate > 0:
                model.add(Dropout(dropout_rate))
            neurons=neurons //2

    model.add(Dense(1, activation='linear'))
    model.compile(loss='mean_squared_error', optimizer=optimizer)
    return model

# Criar o modelo KerasRegressor
model = KerasRegressor(build_fn=create_model, epochs=25, batch_size=32, verbose=1)

# Definir os hiperparâmetros a serem testados
parameters = {
    'neurons': [128,256],
    'optimizer': ['adam'],
    'layer_type': ['conic','rectangular'],
    'dropout_rate': [0.2,0.5],
    'num_layers': [2,3],  # Adicionado para escolher entre 2 ou 3 camadas
    'callbacks': [[EarlyStopping(monitor='val_loss', patience=2)]]
}

# Criar objeto GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=parameters, scoring='r2', cv=3)

# Treinar o modelo usando a busca em grade
grid_result = grid_search.fit(X_train, y_train, validation_data=(X_val, y_val))

# Exibir os melhores hiperparâmetros encontrados
print("Melhores Hiperparâmetros:", grid_result.best_params_)

# Avaliar o modelo no conjunto de teste
test_score = grid_result.score(X_test, y_test)
print(f"Desempenho no Conjunto de Teste: {test_score:.4f}")



/var/folders/m3/zkz1w1516xd9qw8bldx6sp780000gn/T/ipykernel_7287/1241221367.py:28: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasRegressor(build_fn=create_model, epochs=25, batch_size=32, verbose=1)


Epoch 1/25
286/286 [==============================] - 2s 4ms/step - loss: 269.9479 - val_loss: 228.9121
Epoch 2/25
286/286 [==============================] - 1s 3ms/step - loss: 228.4473 - val_loss: 218.5089
Epoch 3/25
286/286 [==============================] - 1s 3ms/step - loss: 211.9869 - val_loss: 210.6485
Epoch 4/25
286/286 [==============================] - 1s 3ms/step - loss: 198.9236 - val_loss: 198.6761
Epoch 5/25
286/286 [==============================] - 1s 3ms/step - loss: 187.8943 - val_loss: 197.4998
Epoch 6/25
286/286 [==============================] - 1s 3ms/step - loss: 179.9882 - val_loss: 185.9579
Epoch 7/25
286/286 [==============================] - 1s 3ms/step - loss: 172.2811 - val_loss: 192.9575
Epoch 8/25
286/286 [==============================] - 1s 3ms/step - loss: 165.5193 - val_loss: 184.1445
Epoch 9/25
286/286 [==============================] - 1s 3ms/step - loss: 160.8766 - val_loss: 182.7797
Epoch 10/25
286/286 [==============================] - 1s 3ms/st

### Convolutional Neural Network (CNN)

Para o CNN realizamos um modelo baseline, uma vez que devido ao poder computacional que possuimos o uso de hiperparametros seria demasiado demorado.

Se o indicado anteriormente não fosse um problema, para a arquitetura de CNN, a estratégia passaria por permitir escolher o kernel_size, bem como o numero de camadas Conv/pooling (maximo de 3), além disso após a realização do flatten a DNN também poderia ser realizada com um máximo de 3 camadas.

Além disso verificamos o comportamento com taza de dropout de 0.2 e 0.5 bem como eraling stopping de 2.

In [5]:
from sklearn.metrics import r2_score
from keras.optimizers import Adam
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(1, activation='linear'))

model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.001))

history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# Evaluate the model
y_pred = model.predict(X_test)
r2 = r2_score(y_test, y_pred)
print("R-squared score on test data: ", r2)

/Users/tiago_silva/opt/anaconda3/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/10
429/429 [==============================] - 14s 30ms/step - loss: 259.2995 - val_loss: 242.5519
Epoch 2/10
429/429 [==============================] - 11s 26ms/step - loss: 246.2193 - val_loss: 244.3100
Epoch 3/10
429/429 [==============================] - 11s 25ms/step - loss: 245.2748 - val_loss: 242.4518
Epoch 4/10
429/429 [==============================] - 14s 33ms/step - loss: 244.0597 - val_loss: 240.9978
Epoch 5/10
429/429 [==============================] - 13s 30ms/step - loss: 244.1934 - val_loss: 240.8664
Epoch 6/10
429/429 [==============================] - 11s 25ms/step - loss: 243.1860 - val_loss: 240.1744
Epoch 7/10
429/429 [==============================] - 10s 24ms/step - loss: 243.1309 - val_loss: 241.7224
Epoch 8/10
429/429 [==============================] - 11s 24ms/step - loss: 243.1982 - val_loss: 239.1278
Epoch 9/10
429/429 [==============================] - 10s 24ms/step - loss: 242.8004 - val_loss: 240.7306
Epoch 10/10
92/92 [===========================

O valor obtido de r2 é inferior ao obtido pela arquitetura do DNN, apresentado a cima. Mas tambem queremos realçar que o número de epochs é bastante inferior e achamos que caso fosse possivel computacionalmente realizar o codigo abaixo, que o valor de r2 seria superior.

In [ ]:
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

# Função para criar o modelo CNN
def create_cnn_model(kernel_size=3, filters=32, dense_units=128, dense_layers=2, dropout_rate=0.5, num_layers=3):
    model = Sequential()
    model.add(Conv1D(filters, kernel_size=kernel_size, activation='relu', input_shape=(X_train.shape[1], 1)))
    model.add(MaxPooling1D(pool_size=2))
    filters=filters //2
    for _ in range(num_layers - 1):  # Adiciona camadas extras
        model.add(Conv1D(filters, kernel_size=kernel_size, activation='relu'))
        model.add(MaxPooling1D(pool_size=2))
        filters=filters //2
    model.add(Flatten())

    for _ in range(dense_layers):  # Adiciona camadas densas
        model.add(Dense(dense_units, activation='relu'))
        model.add(Dropout(dropout_rate))

    model.add(Dense(1, activation='linear'))
    model.compile(loss='mean_squared_error', optimizer=Adam(), metrics=['mean_squared_error'])
    return model

# Crie um modelo KerasRegressor
model = KerasRegressor(build_fn=create_cnn_model, epochs=25, batch_size=32, verbose=1)

# Defina os hiperparâmetros a serem testados
parameters = {
    'kernel_size': [3,5],
    'filters': [32,64],
    'dense_units': [128,256],
    'dense_layers': [2,3],  # Adicionado para escolher entre 2 ou 3 camadas densas
    'dropout_rate': [0.2,0.5],
    'num_layers': [2,3],
    'callbacks': [[EarlyStopping(monitor='val_loss', patience=2)]]
}

# Crie um objeto GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=parameters, scoring='r2', cv=3)

# Treine o modelo usando a busca em grade
grid_result = grid_search.fit(X_train, y_train, validation_data=(X_val, y_val))

# Exiba os melhores hiperparâmetros encontrados
print("Melhores Hiperparâmetros:", grid_result.best_params_)

# Avalie o modelo no conjunto de teste
test_score = grid_result.score(X_test, y_test)
print(f"Desempenho no Conjunto de Teste: {test_score:.4f}")


# Conclusão

Concluímos, então, que o melhor modelo treinado entre todos os que testamos foi o Random Forest, apesar de a arquitetura de DNN apresentar um score bastante semelhante.

Embora saibamos que poderíamos obter melhores resultados nos modelos que utilizam as arquiteturas de DNN e o CNN se tivéssemos mais recursos/poder computacional para os suportar, porque, apesar de ser exequível, demoraria dias para testarmos tudo o que queríamos.

Concluímos também que, apesar de estarmos a usar um dataset que foi utilizado na literatura, conseguimos treinar ou afinar melhor os modelos para que estes apresentassem resultados melhores do que os apresentados no artigo, "DeepSynergy: predicting anti-cancer drug synergy with Deep Learning", em que nos baseamos.

Sabemos também que haveria a possibilidade de obtermos melhores resultados, através do já mencionado maior poder computacional, e se tivéssemos mais combinações de drogas para treinar os modelos.